In [1]:
from graph_utils import *

In [5]:
people = pd.read_csv("data/SDFB/people.csv")
relations = pd.read_csv("data/SDFB/relations.csv")

/var/folders/vb/kwlfq17j5c91v6twzjz7s1n80000gn/T/ipykernel_61965/2453539770.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  relations = pd.read_csv("data/SDFB/relations.csv")


In [11]:
people = people[["id", "display_name"]]
relations = relations[["id", "person1_index", "person2_index"]]